In [0]:
import time

start_time = time.time()

In [0]:
!pip install --upgrade pyspark==3.4.1 spark-nlp==5.4.0


In [0]:
 #Spark NLP Initialization

try:
    import sparknlp
    spark = sparknlp.start()
    print("✅ Spark NLP started successfully")

except ModuleNotFoundError as e:
    raise ModuleNotFoundError(
        "❌ Spark NLP not found.\n"
        "👉 Run: pip install spark-nlp\n"
        "👉 Restart runtime after install."
    ) from e


except RuntimeError as e:
    if "Spark Connect server and Spark master cannot be configured together" in str(e):
        raise RuntimeError(
            "❌ Spark Connect conflict detected.\n"
            "👉 Do NOT call sparknlp.start() in Databricks.\n"
            "👉 Use existing SparkSession instead."
        ) from e
    else:
        raise


except Exception as e:
    if "Java gateway process exited" in str(e):
        raise RuntimeError(
            "❌ Java/JVM failed to start.\n"
            "👉 Ensure Java 8 or 11 is installed.\n"
            "👉 Restart the runtime."
        ) from e


    elif "spark.jsl.settings.pretrained.cache_folder" in str(e):
        raise RuntimeError(
            "❌ Spark NLP cache configuration error.\n"
            "👉 Do NOT set spark.jsl.settings.pretrained.cache_folder manually.\n"
            "👉 Use environment variable SPARK_NLP_CACHE instead."
        ) from e


    elif "OutOfMemoryError" in str(e):
        raise RuntimeError(
            "❌ Out of Memory error.\n"
            "👉 Reduce batch size or increase driver memory."
        ) from e

    else:
        raise RuntimeError(f"❌ Spark NLP init failed: {e}") from e


In [0]:
from sparknlp.annotator import ClassifierDLModel
from pyspark.ml import Pipeline
from pyspark.sql.functions import col


In [0]:
# Imports

try:
    from sparknlp.base import DocumentAssembler
    from sparknlp.annotator import (
        Tokenizer,
        UniversalSentenceEncoder,
        SentimentDLModel
    )
    from pyspark.ml import Pipeline
    from pyspark.sql.functions import col, current_timestamp

    print("✅ Imports successful")

except ModuleNotFoundError as e:
    if "sparknlp" in str(e):
        raise ModuleNotFoundError(
            "❌ Spark NLP not installed.\n"
            "👉 Fix: pip install spark-nlp\n"
            "👉 Restart runtime after installation."
        ) from e
    elif "pyspark" in str(e):
        raise ModuleNotFoundError(
            "❌ PySpark not installed.\n"
            "👉 Fix: pip install pyspark\n"
            "👉 Restart runtime."
        ) from e
    else:
        raise


except ImportError as e:
    if "UniversalSentenceEncoder" in str(e):
        raise ImportError(
            "❌ UniversalSentenceEncoder not available.\n"
            "👉 Spark NLP version too old.\n"
            "👉 Upgrade: pip install --upgrade spark-nlp"
        ) from e
    elif "SentimentDLModel" in str(e):
        raise ImportError(
            "❌ SentimentDLModel not found.\n"
            "👉 Spark NLP ML annotators not available in this version.\n"
            "👉 Upgrade Spark NLP."
        ) from e
    else:
        raise

except Exception as e:
    if "JavaPackage" in str(e):
        raise RuntimeError(
            "❌ Spark NLP JVM classes not loaded.\n"
            "👉 Spark session not initialized correctly.\n"
            "👉 Ensure sparknlp.start() ran successfully."
        ) from e
    else:
        raise RuntimeError(f"❌ Import failed: {e}") from e


In [0]:
df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("/content/sliver_layer__1_ (1).csv")
)

In [0]:
df.printSchema()


In [0]:
from pyspark.sql.functions import col


try:
    if "cleaned_text" not in df.columns and "clean_text" in df.columns:
        df = df.withColumnRenamed("clean_text", "cleaned_text")

except AttributeError:
    raise TypeError("df is not a Spark DataFrame")



REQUIRED_COL = "cleaned_text"


def validate_input_data(df):

    if REQUIRED_COL not in df.columns:
        raise ValueError(f"Missing column: {REQUIRED_COL}")


    if df.rdd.isEmpty():
        raise ValueError(" DataFrame is empty")


    null_count = df.filter(col(REQUIRED_COL).isNull()).count()
    if null_count > 0:
        print(f" {null_count} null rows dropped")

    return df.filter(col(REQUIRED_COL).isNotNull())



try:
    df = validate_input_data(df)

except ValueError as e:
    raise ValueError(e)

except Exception:
    raise RuntimeError(" Spark job failed during validation")



try:
    df.select("cleaned_text").show(5, truncate=False)
except Exception:
    raise RuntimeError("❌ Unable to show DataFrame (Spark issue)")


In [0]:
# ================================
# NLP Components
# ================================
try:
    document_assembler = DocumentAssembler() \
        .setInputCol("clean_text") \
        .setOutputCol("document")

    tokenizer = Tokenizer() \
        .setInputCols(["document"]) \
        .setOutputCol("token")

    embeddings = UniversalSentenceEncoder.pretrained(
        "tfhub_use", "en"
    ).setInputCols(["document"]) \
     .setOutputCol("embeddings")

    sentiment_model = SentimentDLModel.pretrained(
        "sentimentdl_use_twitter", "en"
    ).setInputCols(["embeddings"]) \
     .setOutputCol("sentiment")

    print("NLP components initialized")

except AttributeError:
    raise RuntimeError("Spark NLP not initialized or Spark session missing")

except ValueError:
    raise RuntimeError("Invalid input or output column configuration")

except Exception as e:
    if "Model not found" in str(e):
        raise RuntimeError("Pretrained model not available or download failed")
    elif "No such file or directory" in str(e):
        raise RuntimeError("Spark NLP cache or filesystem issue")
    elif "Java gateway process exited" in str(e):
        raise RuntimeError("JVM or Java version issue")
    else:
        raise RuntimeError("NLP component initialization failed")


In [0]:
df = df.withColumnRenamed("cleaned_text", "clean_text")


In [0]:
# ================================
# Run Prediction
# ================================
try:
    model = pipeline.fit(df)
    prediction_df = model.transform(df)

    print("Sentiment prediction completed")

except ValueError:
    raise RuntimeError("Invalid pipeline configuration or input data")

except AttributeError:
    raise RuntimeError("Pipeline or DataFrame not initialized")

except Exception as e:
    if "Job aborted" in str(e):
        raise RuntimeError("Spark job failed during model execution")
    elif "OutOfMemoryError" in str(e):
        raise RuntimeError("Insufficient memory during prediction")
    elif "AnalysisException" in str(e):
        raise RuntimeError("Schema or column mismatch during prediction")
    else:
        raise RuntimeError("Prediction step failed")


In [0]:
# ================================
# Extract Sentiment Label
# ================================
try:
    final_df = (
        prediction_df
        .withColumn("sentiment_label", col("sentiment")[0]["result"])
        .withColumn("_prediction_timestamp", current_timestamp())
    )

    final_df.select(
        "clean_text",
        "sentiment_label"
    ).show(10, truncate=False)

    print("Sentiment label extracted")

except AttributeError:
    raise RuntimeError("Prediction DataFrame not available")

except IndexError:
    raise RuntimeError("Sentiment result is empty or missing")

except Exception as e:
    if "AnalysisException" in str(e):
        raise RuntimeError("Sentiment column structure mismatch")
    elif "Job aborted" in str(e):
        raise RuntimeError("Spark job failed during sentiment extraction")
    else:
        raise RuntimeError("Sentiment extraction failed")


In [0]:
final_df.printSchema()

In [0]:

# Writing the csv file

try:
    final_df \
        .select("clean_text", "sentiment_label") \
        .coalesce(1) \
        .write \
        .mode("overwrite") \
        .option("header", "true") \
        .csv("/content/final_ml_predictions")

    print("✅ CSV written successfully")

except Exception as e:
    raise RuntimeError(f"❌ CSV write failed: {e}")


In [0]:
final_df.count()

In [0]:
end_time = time.time()

print(f"Total runtime: {round(end_time - start_time, 2)} seconds")